In [7]:
import fitz  # PyMuPDF
import base64
from io import BytesIO

# Open the PDF
pdf_file = 'company profile.pdf'
doc = fitz.open(pdf_file)

# Iterate through the pages
for page_num in range(len(doc)):
    page = doc.load_page(page_num)
    
    # Extract text
    text = page.get_text("text")
    print(f"Page {page_num + 1} text:")
    print(text)

    # Iterate through the images in the page
    images = page.get_images(full=True)
    for img_index, img in enumerate(images):
        xref = img[0]  # xref is the reference to the image
        base_image = doc.extract_image(xref)
        image_bytes = base_image["image"]

        # Convert the image bytes to base64
        image_base64 = base64.b64encode(image_bytes).decode('utf-8')

        # Include the base64 image string with your text
        print(f"Image {img_index + 1} base64: {image_base64}")


Page 1 text:
HAUZ KHAS, DELHI
CARNOTRESEARCH.COM
contact@carnotresearh.om
contac
t@carnot
researh.om
COMPANY PROFILE

Image 1 base64: ...
Image 2 base64: iVBORw0KGgoAAAANSUhEUgAAAHEAAACDCAIAAAA1RmOVAAAACXBIWXMAAA7EAAAOxAGVKw4bAAAgKUlEQVR4nO2dB1gbV7r3halONptyN2Vzd/d+e2+++2VTHJtujLtNb8ZgwGDHiZM4ZTfuICGBG8aARLFjGxvcqC6JTccNd3oXRQUJBKigNpJmRg3MPvOdGRk7G4I3smPHCXqf/3Oeo5Fm5sxP72mjOa9IJItZzGIWs9hvzLDv2S9dlt+EAY6wTmcCOqBQ9WlQgYXsYxqs1wOarOHhus7u3qERALRPjioUFqyPYTJYK4Kgc9VXNyakHsg/X9vDB0BbBPIOCO2xOOyjGU8KcaSaxIzcd5eEzglY83kio6i6roUvBUBZclipG7M0smYbR6ToFcq3JB/8o4vf7+d4v7UsyvvTuLj9J0tudXKVeLPAV2t5FqxmGVco5w3LNu7OetU9yHaWn/0Hvq+4B78TuD5yW2r66aoa5gAA2jgMMUX6X7qkvx5ji5Q8EbR536E/Loi0c1np4BL6vFvoc86BL7kGOoVu2LD70IU7XQDola4BvlIDjY7+6psCIYxx1eO90Gi7Qt8h1zIVaBeQEpmsbiV6Xz3Q9/JytFuBqwu8lKM9MqRXrmFBmi4l1KWAuyF9hwRmSjVfpx59fWmMlfNKkuMKO7cIe9cw0rs+th/4/NV7bdDXe5JOlV7tFvBQFAAVIVopgVWlwoZluoERmDWi7JWpupWaTgXaqdR3KEc7laMd0CgTMnQptd142ZAeUxkUKFM1ISXKlKNdUi0QyOAvVWgXKDleeF2nXNcm07WNaFlqIx+9K/lZvkgFhrHV2jaxCmDthE

In [2]:
import requests
import os
from PIL import Image
import io
from openai import OpenAI
import fitz

# Function to upload the image to imgbb and get the URL
def upload_image(image_bytes, image_format):
    # Save the image as a temporary file
    image = Image.open(io.BytesIO(image_bytes))
    temp_filename = f"temp_image.{image_format.lower()}"
    image.save(temp_filename)

    # Define the imgbb API endpoint and your API key
    imgbb_url = "https://api.imgbb.com/1/upload"
    imgbb_api_key = "5f9225859e745656212ea7fd2d79cc6f"  # Replace this with your imgbb API key

    # Upload the image to imgbb
    with open(temp_filename, "rb") as image_file:
        response = requests.post(
            imgbb_url,
            params={"key": imgbb_api_key},
            files={"image": image_file}
        )
    
    # Parse the response to get the URL
    if response.status_code == 200:
        image_url = response.json()["data"]["url"]
        return image_url
    else:
        raise Exception(f"Error uploading image: {response.status_code}, {response.text}")

# Function to interact with the OpenAI API using the image URL
def send_image_to_openai(image_url):
    client = OpenAI()

    # Send the image URL to GPT-4
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": "What’s in this image?"},
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": image_url,
                        },
                    },
                ],
            }
        ],
        max_tokens=300,
    )

    # Return the response
    return response.choices[0]

# Open the PDF and extract images
pdf_file = 'company profile.pdf'
doc = fitz.open(pdf_file)

# Iterate through the pages
for page_num in range(len(doc)):
    page = doc.load_page(page_num)
    
    # Extract text
    text = page.get_text("text")
    print(f"Page {page_num + 1} text:")
    print(text)

    # Iterate through the images in the page
    images = page.get_images(full=True)
    for img_index, img in enumerate(images):
        xref = img[0]  # xref is the reference to the image
        base_image = doc.extract_image(xref)
        image_bytes = base_image["image"]
        image_ext = base_image["ext"]  # Get the image extension (e.g., 'png', 'jpeg')

        # Upload the image and get the URL
        try:
            image_url = upload_image(image_bytes, image_ext)
            print(f"Image {img_index + 1} uploaded: {image_url}")

            # Send the image URL to OpenAI GPT-4
            caption = send_image_to_openai(image_url)
            print(f"Image {img_index + 1} caption: {caption}")
        
        except Exception as e:
            print(f"Error handling image {img_index + 1}: {e}")

    print(f"Final Page {page_num + 1} content:\n{text}")



Page 1 text:
HAUZ KHAS, DELHI
CARNOTRESEARCH.COM
contact@carnotresearh.om
contac
t@carnot
researh.om
COMPANY PROFILE

Image 1 uploaded: https://i.ibb.co/QDyRHmH/temp-image.jpg
Error handling image 1: 'ChatCompletionMessage' object is not subscriptable


KeyboardInterrupt: 

In [5]:
from langchain_google_community import GoogleDriveLoader

In [11]:
loader = GoogleDriveLoader(
    folder_id="1yucgL9WGgWZdM1TOuKkeghlPizuzMYb5",
    token_path="google token.json",
    # Optional: configure whether to recursively fetch files from subfolders. Defaults to False.
    recursive=False,
)

In [12]:
data = loader.load()

ValueError: Authorized user info was not in the expected format, missing fields client_id, client_secret, refresh_token.

In [ ]:
payload = {
        "pipelineTasks": [
            {
                "taskType": "tts",
                "config": {
                    "language": {
                        "sourceLanguage": "hi",
                    }
                }
            }
        ],
        "pipelineRequestConfig": {
            "pipelineId" : "64392f96daac500b55c543cd"
        }
    }

headers = {
        "Content-Type": "application/json",
        "userID": "e832f2d25d21443e8bb90515f1079041",
        "ulcaApiKey": "39e27ce432-f79c-46f8-9c8c-c0856007cb4b"
    }

response = requests.post('https://meity-auth.ulcacontrib.org/ulca/apis/v0/model/getModelsPipeline', json=payload, headers=headers)

if response.status_code == 200:
        response_data = response.json()
        service_id = response_data["pipelineResponseConfig"][0]["config"][0]["serviceId"]

        compute_payload = {
    "pipelineTasks": [       
        {
            "taskType": "tts",
            "config": {
                "language": {
                    "sourceLanguage": "gu"
                },
                "serviceId": "",
                "gender": "female",
                "samplingRate": 8000
            }
        }
    ],
    "inputData": {
        "input": [
            {
                "source": "મારું નામ વિહીર છે અને હું ભાષાવર્ષનો ઉપયોગ કરી રહ્યો છું"
            }
        ]
    }
}

callback_url = response_data["pipelineInferenceAPIEndPoint"]["callbackUrl"]
        
headers2 = {
            "Content-Type": "application/json",
            response_data["pipelineInferenceAPIEndPoint"]["inferenceApiKey"]["name"]:
                response_data["pipelineInferenceAPIEndPoint"]["inferenceApiKey"]["value"]
        }

compute_response = requests.post(callback_url, json=compute_payload, headers=headers2)

if compute_response.status_code == 200:
            compute_response_data = compute_response.json()
            translated_content = compute_response_data["pipelineResponse"][0]["output"][0]["target"]
            print(translated_content)
